# 🚀 D2L Proje Yapısı - Demo

Bu notebook, projenin nasıl kullanılacağını gösterir:
- ✅ Python modüllerini kullanma
- ✅ CUDA kernellerini test etme
- ✅ Görselleştirme araçları
- ✅ Eğitim döngüsü örneği

## 1. Import'lar

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# D2L özel modüller
from d2l_custom.utils import Timer, try_gpu, gpu_info
from d2l_custom.models import LinearRegression
from d2l_custom.data import synthetic_data, load_data
from d2l_custom.visualization import plot, plot_training_curve
from d2l_custom.training import train

# CUDA operasyonları (derlenmişse)
try:
    from d2l_custom.cuda_ops import custom_matmul, custom_relu
    CUDA_AVAILABLE = True
except ImportError:
    print("⚠️  CUDA modülleri derli değil. PyTorch fallback kullanılacak.")
    CUDA_AVAILABLE = False

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Sentetik Veri Üretimi

In [ ]:
# Doğrusal regresyon için sentetik veri
true_w = torch.tensor([2.0, -3.4, 5.1])
true_b = 4.2
X, y = synthetic_data(true_w, true_b, num_examples=1000)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"True weights: {true_w.tolist()}")
print(f"True bias: {true_b}")

## 3. DataLoader Oluşturma

In [ ]:
train_loader, test_loader = load_data(X, y, batch_size=32, train_ratio=0.8)
print(f"Train batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

## 4. Model Oluşturma

In [ ]:
model = LinearRegression(in_features=3)
device = try_gpu()
model.to(device)

loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.03)

print(f"Model: {model}")
print(f"Device: {device}")

## 5. Eğitim

In [ ]:
history = train(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    num_epochs=10,
    device=device
)

## 6. Görselleştirme

In [ ]:
plot_training_curve(
    train_losses=history['train_loss'],
    val_losses=history['test_loss']
)

## 7. Sonuçlar

In [ ]:
# Öğrenilen parametreleri göster
learned_w = model.net.weight.data.squeeze()
learned_b = model.net.bias.data.item()

print("═" * 50)
print("Gerçek vs Öğrenilen Parametreler")
print("═" * 50)
print(f"True w:    {true_w.tolist()}")
print(f"Learned w: {learned_w.tolist()}")
print(f"")
print(f"True b:    {true_b:.3f}")
print(f"Learned b: {learned_b:.3f}")
print("═" * 50)

## 8. CUDA Kernelleri (Opsiyonel)

Eğer CUDA modülleri derlenmişse, performans karşılaştırması yapabiliriz.

In [ ]:
if CUDA_AVAILABLE and torch.cuda.is_available():
    N = 1024
    A = torch.randn(N, N, device='cuda')
    B = torch.randn(N, N, device='cuda')
    
    # PyTorch matmul
    timer = Timer()
    timer.start()
    for _ in range(100):
        C_torch = torch.mm(A, B)
    torch.cuda.synchronize()
    time_torch = timer.stop()
    
    # Custom CUDA matmul
    timer.start()
    for _ in range(100):
        C_cuda = custom_matmul(A, B)
    torch.cuda.synchronize()
    time_cuda = timer.stop()
    
    print(f"PyTorch matmul: {time_torch:.3f}s")
    print(f"Custom CUDA matmul: {time_cuda:.3f}s")
    print(f"Speedup: {time_torch/time_cuda:.2f}x")
    
    # Doğruluk kontrolü
    error = (C_torch - C_cuda).abs().max().item()
    print(f"Max error: {error:.2e}")
else:
    print("⚠️  CUDA mevcut değil veya modüller derlenmemiş.")

## 🎉 Tebrikler!

D2L proje yapısını başarıyla kullandınız. Şimdi:

1. **Roadmap'e** göz atın: `notebooks/0_roadmap/roadmap.md`
2. **Bölümleri** inceleyin: `notebooks/01_introduction/`, `02_preliminaries/`, vs.
3. **CUDA kernellerini** derleyin: `make cuda-build`
4. **Testleri** çalıştırın: `make test`

İyi çalışmalar! 🚀